In [2]:
import os
import numpy as np
import h5py
import torch
import scipy.io as sio

In [3]:
# --- 定数の設定 ---
# .matのtargetデータフォルダ場所
MAT_TARGET_PATH = "/mnt/c/Users/onion/Documents/data"
# .ptのtargetデータフォルダ保存場所
PT_TARGET_PATH = "/mnt/c/Users/onion/Documents/processed_data/target"
# targetファイル名
TARGET_NAMES = ["m_train", "m_val", "m_test", "kappa_train", "kappa_val", "kappa_test"]



MAT_PATH = "/mnt/c/Users/onion/Documents/data/structures_train"
PT_PATH = "/mnt/c/Users/onion/Documents/processed_data/train"

In [4]:
# ---keyの取得 ---
for TARGET_NAME in TARGET_NAMES:
    mat_target_path = os.path.join(MAT_TARGET_PATH, f"{TARGET_NAME}.mat")

    with h5py.File(mat_target_path, "r") as f:
        keys = list(f.keys())
        print(keys)


['y']
['y']
['y']
['y']
['y']
['y']


In [ ]:
# --- keyの設定 ---
# 上記のコードから取得したキーからほしいデータだけを指定する
TARGET_KEYS = ['y', 'y', 'y', 'y', 'y', 'y']

In [ ]:
# --- targetのmat2pt ---
# pathのディレクトリ部分が存在しない場合は作成、存在してもエラーにしない
os.makedirs(os.path.dirname(PT_TARGET_PATH), exist_ok=True)

for TARGET_KEY in TARGET_KEYS:
    for TARGET_NAME in TARGET_NAMES:
        mat_target_path = os.path.join(MAT_TARGET_PATH, f"{TARGET_NAME}.mat")
        pt_target_path = os.path.join(PT_TARGET_PATH, f"{TARGET_NAME}.pt")

        # MATLABファイルを読み込み、指定された変数をPyTorchのテンソルに変換して保存
        with h5py.File(mat_target_path, "r") as f:
            data = f[TARGET_KEY]
            data = np.array(data).T
            data = torch.tensor(data)
            torch.save(data, pt_target_path)

            if os.path.exists(f"{pt_target_path}"):
                print(f"{pt_target_path} が作成されました。")

/mnt/c/Users/onion/Documents/processed_data/target/m_train.pt が作成されました。
/mnt/c/Users/onion/Documents/processed_data/target/m_val.pt が作成されました。
/mnt/c/Users/onion/Documents/processed_data/target/m_test.pt が作成されました。
/mnt/c/Users/onion/Documents/processed_data/target/kappa_train.pt が作成されました。
/mnt/c/Users/onion/Documents/processed_data/target/kappa_val.pt が作成されました。
/mnt/c/Users/onion/Documents/processed_data/target/kappa_test.pt が作成されました。


In [ ]:
# --- structureのmat2pt ---
os.makedirs(os.path.dirname(PT_PATH), exist_ok=True)
num_mat_files = len(os.listdir(MAT_PATH))

for i in range(5):
    mat_path = os.path.join(MAT_PATH, f"structure_{i+1}.mat")
    pt_path = os.path.join(PT_PATH, f"structure_{i+1}.pt")

    data = sio.loadmat(mat_path)
    key = data.keys()
    data = data['BW']

    # data = np.array(data).T
    # data = torch.tensor(data)
    # torch.save(data, pt_path)